In [1]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.datasets import UD_ENGLISH
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings
from flair.data import Dictionary
from flair import Seque

/home/anselmi/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
# 1. create the corpus

columns = {0: 'text', 1: 'tag'}
data_folder = '/home/anselmi/Desktop/Courses/Data Science Project'
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='tag_test.txt',
                              test_file='tag_test.txt',
                              dev_file='tag_test.txt')

2020-11-05 10:17:16,721 Reading data from /home/anselmi/Desktop/Courses/Data Science Project
2020-11-05 10:17:16,724 Train: /home/anselmi/Desktop/Courses/Data Science Project/tag_test.txt
2020-11-05 10:17:16,725 Dev: /home/anselmi/Desktop/Courses/Data Science Project/tag_test.txt
2020-11-05 10:17:16,727 Test: /home/anselmi/Desktop/Courses/Data Science Project/tag_test.txt


In [3]:
# 2. what tag do we want to predict?
tag_type = 'ner'

In [42]:
# 3. make the tag dictionary from the corpus

# tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
# print(tag_dictionary)

tag_dictionary = Dictionary()
print(tag_dictionary)

help(Dictionary)

Dictionary with 1 tags: <unk>
Help on class Dictionary in module flair.data:

class Dictionary(builtins.object)
 |  Dictionary(add_unk=True)
 |  
 |  This class holds a dictionary that maps strings to IDs, used to generate one-hot encodings of strings.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, add_unk=True)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __len__(self) -> int
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  add_item(self, item: str) -> int
 |      add string - if already in dictionary returns its ID. if not in dictionary, it will get a new ID.
 |      :param item: a string for which to assign an id.
 |      :return: ID of string
 |  
 |  get_idx_for_item(self, item: str) -> int
 |      returns the ID of the string, otherwise 0
 |      :param item: string for which ID is requested
 |      :return: ID of string, otherwise 0
 |  
 |  get_idx_for_items(self, items: List[str]) -> List[int]
 |      returns the IDs for each

In [3]:
from flair.data import Dictionary
help(Corpus)

Help on class Corpus in module flair.data:

class Corpus(builtins.object)
 |  Corpus(train: flair.data.FlairDataset, dev: flair.data.FlairDataset = None, test: flair.data.FlairDataset = None, name: str = 'corpus')
 |  
 |  Methods defined here:
 |  
 |  __init__(self, train: flair.data.FlairDataset, dev: flair.data.FlairDataset = None, test: flair.data.FlairDataset = None, name: str = 'corpus')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __str__(self) -> str
 |      Return str(self).
 |  
 |  downsample(self, percentage: float = 0.1, downsample_train=True, downsample_dev=True, downsample_test=True)
 |  
 |  filter_empty_sentences(self)
 |  
 |  filter_long_sentences(self, max_charlength: int)
 |  
 |  get_all_sentences(self) -> torch.utils.data.dataset.Dataset
 |  
 |  get_label_distribution(self)
 |  
 |  make_label_dictionary(self, label_type: str = None) -> flair.data.Dictionary
 |      Creates a dictionary of all labels assigned to the sentences 

2020-11-05 09:54:48,091 Reading data from /home/anselmi/Desktop/Courses/Data Science Project
2020-11-05 09:54:48,093 Train: /home/anselmi/Desktop/Courses/Data Science Project/tag_test.txt
2020-11-05 09:54:48,095 Dev: /home/anselmi/Desktop/Courses/Data Science Project/tag_test.txt
2020-11-05 09:54:48,098 Test: /home/anselmi/Desktop/Courses/Data Science Project/tag_test.txt


In [9]:
d = corpus.make_tag_dictionary('ner')

In [10]:
print(d)

Dictionary with 5 tags: <unk>, O, , <START>, <STOP>


In [4]:
corpus.train[0]

Sentence: "This sentence has propaganda tags for all but the first and last two words"   [− Tokens: 14  − Token-Labels: "This <0> sentence <1> has <1> propaganda <1> tags <1> for <1> all <1> but <1> the <1> first <1> and <1> last <1> two <0> words <0>"]

In [6]:
corpus.train[0].to_tagged_string()

'This is an example <B-STRAWMAN> sentence'

In [9]:
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, StackedEmbeddings, TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.data import Sentence

In [27]:
# Dataset columns
columns = {0 : 'text', 1 : 'ner'}

# Data path
data_path = './data/'

# Create corpus
corpus = ColumnCorpus(data_path, columns,
                              train_file = 'train.txt',
                              #test_file = 'test.txt',
                              dev_file = 'dev.txt')

# remove empty sentences
corpus.filter_empty_sentences()

2020-11-11 20:47:03,883 Reading data from data
2020-11-11 20:47:03,884 Train: data/train.txt
2020-11-11 20:47:03,885 Dev: data/dev.txt
2020-11-11 20:47:03,886 Test: None
2020-11-11 20:47:09,982 Filtering empty sentences
2020-11-11 20:47:10,016 Corpus: 12830 train + 2033 dev + 1426 test sentences


In [28]:
tag_type = 'ner'

tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

In [33]:
embedding_types = [
    #WordEmbeddings('glove'),
    TransformerWordEmbeddings('bert-base-uncased')
    # More stuff maybe
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

In [34]:
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionary,
                        tag_type=tag_type,
                        use_crf=True)

In [35]:
trainer = ModelTrainer(tagger, corpus)

trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2020-11-11 21:02:46,146 ----------------------------------------------------------------------------------------------------
2020-11-11 21:02:46,149 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): TransformerWordEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear

2020-11-11 21:02:46,151 ----------------------------------------------------------------------------------------------------
2020-11-11 21:02:46,152 Corpus: "Corpus: 12830 train + 2033 dev + 1426 test sentences"
2020-11-11 21:02:46,153 ----------------------------------------------------------------------------------------------------
2020-11-11 21:02:46,155 Parameters:
2020-11-11 21:02:46,156  - learning_rate: "0.1"
2020-11-11 21:02:46,158  - mini_batch_size: "32"
2020-11-11 21:02:46,159  - patience: "3"
2020-11-11 21:02:46,160  - anneal_factor: "0.5"
2020-11-11 21:02:46,161  - max_epochs: "150"
2020-11-11 21:02:46,162  - shuffle: "True"
2020-11-11 21:02:46,163  - train_with_dev: "False"
2020-11-11 21:02:46,164  - batch_growth_annealing: "False"
2020-11-11 21:02:46,165 ----------------------------------------------------------------------------------------------------
2020-11-11 21:02:46,166 Model training base path: "resources/taggers/example-ner"
2020-11-11 21:02:46,167 ------------

2020-11-11 21:52:03,490 ----------------------------------------------------------------------------------------------------
2020-11-11 21:52:03,491 EPOCH 5 done: loss 4.0863 - lr 0.1000000
2020-11-11 21:52:25,742 DEV : loss 6.209514617919922 - score 0.3687
Epoch     5: reducing learning rate of group 0 to 5.0000e-02.
2020-11-11 21:52:25,905 BAD EPOCHS (no improvement): 4
2020-11-11 21:52:25,906 ----------------------------------------------------------------------------------------------------
2020-11-11 21:53:04,462 epoch 6 - iter 40/401 - loss 2.99350680 - samples/sec: 33.20 - lr: 0.050000
2020-11-11 21:53:45,949 epoch 6 - iter 80/401 - loss 3.16134791 - samples/sec: 30.86 - lr: 0.050000
2020-11-11 21:54:25,988 epoch 6 - iter 120/401 - loss 3.31509494 - samples/sec: 31.97 - lr: 0.050000
2020-11-11 21:55:07,062 epoch 6 - iter 160/401 - loss 3.25413935 - samples/sec: 31.17 - lr: 0.050000
2020-11-11 21:55:45,646 epoch 6 - iter 200/401 - loss 3.35865395 - samples/sec: 33.18 - lr: 0.0500

2020-11-11 22:31:08,152 epoch 11 - iter 200/401 - loss 2.63263380 - samples/sec: 32.33 - lr: 0.025000
2020-11-11 22:31:50,230 epoch 11 - iter 240/401 - loss 2.62952410 - samples/sec: 30.42 - lr: 0.025000
2020-11-11 22:32:27,941 epoch 11 - iter 280/401 - loss 2.61703288 - samples/sec: 33.95 - lr: 0.025000
2020-11-11 22:33:08,291 epoch 11 - iter 320/401 - loss 2.61018592 - samples/sec: 31.73 - lr: 0.025000
2020-11-11 22:33:47,382 epoch 11 - iter 360/401 - loss 2.60474727 - samples/sec: 32.75 - lr: 0.025000
2020-11-11 22:34:25,745 epoch 11 - iter 400/401 - loss 2.60846058 - samples/sec: 33.37 - lr: 0.025000
2020-11-11 22:34:26,613 ----------------------------------------------------------------------------------------------------
2020-11-11 22:34:26,614 EPOCH 11 done: loss 2.6059 - lr 0.0250000
2020-11-11 22:34:48,811 DEV : loss 3.172679901123047 - score 0.8487
2020-11-11 22:34:48,976 BAD EPOCHS (no improvement): 1
2020-11-11 22:34:48,977 --------------------------------------------------

2020-11-11 23:10:11,510 BAD EPOCHS (no improvement): 2
2020-11-11 23:10:11,511 ----------------------------------------------------------------------------------------------------
2020-11-11 23:10:50,863 epoch 17 - iter 40/401 - loss 2.35697298 - samples/sec: 32.53 - lr: 0.012500
2020-11-11 23:11:30,482 epoch 17 - iter 80/401 - loss 2.42980777 - samples/sec: 32.31 - lr: 0.012500
2020-11-11 23:12:10,841 epoch 17 - iter 120/401 - loss 2.40273140 - samples/sec: 31.72 - lr: 0.012500
2020-11-11 23:12:49,377 epoch 17 - iter 160/401 - loss 2.39606540 - samples/sec: 33.22 - lr: 0.012500
2020-11-11 23:13:30,890 epoch 17 - iter 200/401 - loss 2.40463871 - samples/sec: 30.84 - lr: 0.012500
2020-11-11 23:14:10,793 epoch 17 - iter 240/401 - loss 2.37866832 - samples/sec: 32.08 - lr: 0.012500
2020-11-11 23:14:50,091 epoch 17 - iter 280/401 - loss 2.35118823 - samples/sec: 32.58 - lr: 0.012500
2020-11-11 23:15:31,042 epoch 17 - iter 320/401 - loss 2.32836643 - samples/sec: 31.26 - lr: 0.012500
2020-1

2020-11-11 23:50:49,733 epoch 22 - iter 320/401 - loss 2.14787780 - samples/sec: 32.13 - lr: 0.006250
2020-11-11 23:51:28,015 epoch 22 - iter 360/401 - loss 2.15963760 - samples/sec: 33.44 - lr: 0.006250
2020-11-11 23:52:08,974 epoch 22 - iter 400/401 - loss 2.16449504 - samples/sec: 31.25 - lr: 0.006250
2020-11-11 23:52:09,892 ----------------------------------------------------------------------------------------------------
2020-11-11 23:52:09,894 EPOCH 22 done: loss 2.1655 - lr 0.0062500
2020-11-11 23:52:32,520 DEV : loss 3.0588812828063965 - score 0.8411
Epoch    22: reducing learning rate of group 0 to 3.1250e-03.
2020-11-11 23:52:32,683 BAD EPOCHS (no improvement): 4
2020-11-11 23:52:32,684 ----------------------------------------------------------------------------------------------------
2020-11-11 23:53:11,386 epoch 23 - iter 40/401 - loss 2.07159922 - samples/sec: 33.08 - lr: 0.003125
2020-11-11 23:53:49,732 epoch 23 - iter 80/401 - loss 1.99369564 - samples/sec: 33.39 - lr:

2020-11-12 00:29:23,130 epoch 28 - iter 80/401 - loss 2.09892673 - samples/sec: 29.68 - lr: 0.001563
2020-11-12 00:30:02,919 epoch 28 - iter 120/401 - loss 2.09451132 - samples/sec: 32.17 - lr: 0.001563
2020-11-12 00:30:44,610 epoch 28 - iter 160/401 - loss 2.12828819 - samples/sec: 30.71 - lr: 0.001563
2020-11-12 00:31:23,736 epoch 28 - iter 200/401 - loss 2.13757448 - samples/sec: 32.72 - lr: 0.001563
2020-11-12 00:32:02,113 epoch 28 - iter 240/401 - loss 2.10396618 - samples/sec: 33.36 - lr: 0.001563
2020-11-12 00:32:39,499 epoch 28 - iter 280/401 - loss 2.10383987 - samples/sec: 34.24 - lr: 0.001563
2020-11-12 00:33:21,100 epoch 28 - iter 320/401 - loss 2.09929715 - samples/sec: 30.77 - lr: 0.001563
2020-11-12 00:34:02,506 epoch 28 - iter 360/401 - loss 2.10753193 - samples/sec: 30.92 - lr: 0.001563
2020-11-12 00:34:42,569 epoch 28 - iter 400/401 - loss 2.09864487 - samples/sec: 31.95 - lr: 0.001563
2020-11-12 00:34:43,512 -----------------------------------------------------------

2020-11-12 01:10:13,907 ----------------------------------------------------------------------------------------------------
2020-11-12 01:10:13,908 EPOCH 33 done: loss 2.0707 - lr 0.0007813
2020-11-12 01:10:36,192 DEV : loss 3.0591745376586914 - score 0.8412
2020-11-12 01:10:36,355 BAD EPOCHS (no improvement): 3
2020-11-12 01:10:36,366 ----------------------------------------------------------------------------------------------------
2020-11-12 01:11:17,801 epoch 34 - iter 40/401 - loss 2.10622119 - samples/sec: 30.90 - lr: 0.000781
2020-11-12 01:11:56,769 epoch 34 - iter 80/401 - loss 2.12725564 - samples/sec: 32.85 - lr: 0.000781
2020-11-12 01:12:36,522 epoch 34 - iter 120/401 - loss 2.13181892 - samples/sec: 32.20 - lr: 0.000781
2020-11-12 01:13:17,069 epoch 34 - iter 160/401 - loss 2.06728183 - samples/sec: 31.57 - lr: 0.000781
2020-11-12 01:13:56,177 epoch 34 - iter 200/401 - loss 2.07436619 - samples/sec: 32.73 - lr: 0.000781
2020-11-12 01:14:37,134 epoch 34 - iter 240/401 - lo

2020-11-12 01:49:17,974 epoch 39 - iter 200/401 - loss 2.08526417 - samples/sec: 30.42 - lr: 0.000195
2020-11-12 01:49:58,110 epoch 39 - iter 240/401 - loss 2.06495566 - samples/sec: 31.90 - lr: 0.000195
2020-11-12 01:50:38,130 epoch 39 - iter 280/401 - loss 2.06642269 - samples/sec: 31.99 - lr: 0.000195
2020-11-12 01:51:17,032 epoch 39 - iter 320/401 - loss 2.06848159 - samples/sec: 32.91 - lr: 0.000195
2020-11-12 01:51:55,372 epoch 39 - iter 360/401 - loss 2.05339388 - samples/sec: 33.39 - lr: 0.000195
2020-11-12 01:52:37,384 epoch 39 - iter 400/401 - loss 2.06171291 - samples/sec: 30.47 - lr: 0.000195
2020-11-12 01:52:38,120 ----------------------------------------------------------------------------------------------------
2020-11-12 01:52:38,121 EPOCH 39 done: loss 2.0616 - lr 0.0001953
2020-11-12 01:53:00,363 DEV : loss 3.0484695434570312 - score 0.8408
2020-11-12 01:53:00,527 BAD EPOCHS (no improvement): 1
2020-11-12 01:53:00,527 -------------------------------------------------

{'test_score': 0.8929,
 'dev_score_history': [0.8526,
  0.8522,
  0.8426,
  0.7658,
  0.3687,
  0.8528,
  0.8528,
  0.847,
  0.8297,
  0.851,
  0.8487,
  0.847,
  0.8443,
  0.8497,
  0.8395,
  0.8482,
  0.8423,
  0.8473,
  0.8484,
  0.8477,
  0.8473,
  0.8411,
  0.8391,
  0.8389,
  0.8404,
  0.8437,
  0.846,
  0.8402,
  0.8406,
  0.8373,
  0.8433,
  0.8426,
  0.8412,
  0.8408,
  0.841,
  0.8419,
  0.8406,
  0.8408,
  0.8408,
  0.8407,
  0.8415,
  0.841],
 'train_loss_history': [11.253379627356209,
  5.916571028809297,
  4.83766245871708,
  4.4146733468310195,
  4.086303799526947,
  3.271066280225863,
  3.1399421614601724,
  3.036928580883435,
  2.922407278990805,
  2.8361931168230394,
  2.605942691650771,
  2.5539224242926237,
  2.4986031158012048,
  2.459933787658625,
  2.3635609848540917,
  2.3147391123961927,
  2.316668475060689,
  2.2674471104977436,
  2.2106180141988836,
  2.2060668597792152,
  2.181502441812929,
  2.1655034605701666,
  2.1403885898447395,
  2.12758843968634,
  2.

In [7]:
model = SequenceTagger.load('./resources/taggers/example-ner/final-model.pt')

2020-11-25 13:44:31,933 loading file ./resources/taggers/example-ner/final-model.pt


In [16]:
s = Sentence('Hitler is a horribe man')
model.predict(s)

In [17]:
s.to_tagged_string()

'Hitler is a horribe man'

In [ ]:
model.